In [1]:
from requests_html import HTMLSession
import json
import time

In [3]:
class Reviews:
    def __init__(self,asin) -> None:
        self.asin = asin
        self.session = HTMLSession()
        self.headers = {'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15'}
        self.url = f'https://www.amazon.co.uk/product-reviews/{self.asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF&reviewerType=all_reviews&sortBy=recent&pageNumber='
    
    def pagination(self,page):
        r = self.session.get(self.url + str(page))
        if not r.html.find('div[data-hook=review]'):
            return False
        else:
            return r.html.find('div[data-hook=review]')
    
    def parse(self, reviews):
        total = []
        for review in reviews:
            title = review.find('a[data-hook=review-title] span', first=True).text
            rating = review.find('i[data-hook=review-star-rating] span', first=True).text
            body = review.find('span[data-hook=review-body] span', first=True).text.replace('\n','').strip()
            
            data = {
                'title':title,
                'rating':rating,
                'body':body[:100]
            }
            total.append(data)
        return total
    
    def save(self, results):
        with open(self.asin + '-reviews.json', 'w') as f:
            json.dump(results, f)
    
if __name__ == '__main__':
    amz = Reviews('B0779B2K8B')
    results = []
    for x in range(1,5):
        print('getting page ',x)
        time.sleep(0.3)
        reviews = amz.pagination(x)
        if reviews is not False:
            results.append(amz.parse(reviews))
        else:
            print('no more pages')
            break
    amz.save(results)

getting page  1
getting page  2
no more pages
